In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory;pip install -e ".[torch,metrics]"
!pip install bitsandbytes>=0.39.0

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15353, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 15353 (delta 191), reused 262 (delta 167), pack-reused 15050
Receiving objects: 100% (15353/15353), 221.76 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (11224/11224), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... - \ | / done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - done
  Preparing editable metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json

file_path = '/kaggle/working/LLaMA-Factory/data/dataset_info.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# configure the MedQA and PubMedQA dataset information 
new_data = {
    "MedQA_train": {"file_name": "MedQA/train.json"},
    "MedQA_test": {"file_name": "MedQA/test.json"},
    "PubMedQA_pqal_train": {"file_name": "PubMedQA/pqal_train_set.json"},
    "PubMedQA_pqal_test": {"file_name": "PubMedQA/pqal_test_set.json"}
}

# update information in dataset_info.json file
data.update(new_data)
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

In [3]:
!ls /kaggle/input/

medqa-pubmedqa


In [4]:
# create MedQA and PubMedQA directories
!mkdir -p /kaggle/working/LLaMA-Factory/data/MedQA
!mkdir -p /kaggle/working/LLaMA-Factory/data/PubMedQA

# copy MedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/MedQA/train.json /kaggle/working/LLaMA-Factory/data/MedQA/
!cp /kaggle/input/medqa-pubmedqa/MedQA/test.json /kaggle/working/LLaMA-Factory/data/MedQA/

# copy PubMedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_train_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_test_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/

In [5]:
%cd LLaMA-Factory

/kaggle/working/LLaMA-Factory


In [6]:
# install packages for quantizaiton
!pip install auto_gptq>=0.5.0
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00


In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_lr=1e-5 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 12:03:03.071695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 12:03:03.071813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 12:03:03.196265: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 12:03:22 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 12:03:22 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 100%|███████████████| 1.29k/1.29k [00:00<00:00, 6.75MB/s]


In [8]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_lr=1e-4 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 12:46:37.850373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 12:46:37.850436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 12:46:37.851932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 12:46:47 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 12:46:47 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2161] 2024-07-13 12:46:47,804 >> loading file vo

In [9]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_bit 8 \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-04 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\qlora\train_lr=5e-4 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2 

2024-07-13 13:30:00.481668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 13:30:00.481746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 13:30:00.483919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 13:30:10 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
07/13/2024 13:30:10 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2161] 2024-07-13 13:30:10,782 >> loading file vo